### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2023-10-14 22:10:48--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.


HTTP request sent, awaiting response... 302 Found
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2023-10-14 22:10:48--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7259e1ce9cf26f5ef096637381.dl.dropboxusercontent.com/cd/0/get/CFmor7mXStPIipn9fyAuTUZptiHiQKjk5YypBT7Sge0fQRP3zRxBIaFwrFV4YF1o92nap6o2VCg6CwkXAE94In-WZrxe2VWQ-m7Mg8EEE7MfBtEe8cb3oad1j5-8W_Fyrcw/file?dl=1# [following]
--2023-10-14 22:10:49--  https://uc7259e1ce9cf26f5ef096637381.dl.dropboxusercontent.com/cd/0/get/CFmor7mXStPIipn9fyAuTUZptiHiQKjk5YypBT7Sge0fQRP3zRxBIaFwrFV4YF1o92nap6o2VCg6CwkXAE94In-WZrxe2VWQ-m7Mg8EEE7MfBtEe8cb3oad1j5-8W_Fyrcw/file?dl=1
Resolving uc7259e1ce9cf26f5ef096637381.dl.dropboxusercontent.com (uc7259e1ce9cf26f5ef096637381.dl.dropboxusercontent.com)... 162.125.71.15, 2620:100:6028:15::a27d:470f
Connecting to uc7259e1ce9cf26f5ef096637381.dl.

,author,day,id,link,month,summary,tag,title,year
19167,"[{'name': 'Eric Neufeld'}, {'name': 'J. D. Hor...",27,1304.1521v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Many writers have observed that default logics...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Conditioning on Disjunctive Knowledge: Default...,2013
10031,"[{'name': 'Shixiang Gu'}, {'name': 'Ethan Holl...",3,1610.00633v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Reinforcement learning holds the promise of en...,"[{'term': 'cs.RO', 'scheme': 'http://arxiv.org...",Deep Reinforcement Learning for Robotic Manipu...,2016
12652,[{'name': 'Liu Liang'}],7,1304.1995v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,Image retrieval has been a top topic in the fi...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Image Retrieval using Histogram Factorization ...,2013
5286,"[{'name': 'Gungor Polatkan'}, {'name': 'Mingyu...",22,1209.5019v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Super-resolution methods form high-resolution ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",A Bayesian Nonparametric Approach to Image Sup...,2012
25417,"[{'name': 'Mohini P. Sardey'}, {'name': 'G. K....",27,1508.06728v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,Basic group of visual techniques such as color...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",A Comparative Analysis of Retrieval Techniques...,2015


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [5]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(x.lower())) for x in lines]

In [6]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [7]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        lexems = line.split(' ')
        lexems = (n - 1)*[UNK] + lexems + [EOS]
        i, m = n - 1, len(lexems)
        while i < m:
            prefix = lexems[i - (n - 1): i]
            pivot = lexems[i]
            counts[tuple(prefix)][pivot] += 1
            i += 1

    
    return counts


In [8]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [9]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities

        for prefix in counts:
            denom = sum(counts[prefix].values())
            for word, word_count in counts[prefix].items():
                self.probs[prefix][word] = word_count / denom

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [10]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [12]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    candidates = [(value, key) for (key, value) in lm.get_possible_next_tokens(prefix).items()]
    if len(candidates) == 0:
        return ' '
    if temperature == 0:
        return candidates[np.argmax([x[0] for x in candidates])][1]

    probs = np.array([x[0]**(1/temperature) for x in candidates])
    words = [x[1] for x in candidates]
    denom = np.sum(probs)
    probs = [x / denom for x in probs]

    return np.random.choice(p=probs, a=words)

In [13]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


In [14]:
get_next_token(lm, 'evolution is', 0)

'considered'

Let's have fun with this model

In [15]:
prefix = 'I love yandex data school and' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

I love yandex data school and office of the individual nn weights ( random round private stochastic gradient langevin dynamics : probabilistic content models in action , expressing these algorithms have become dominant constraints in the original sources of training neural networks ; in this work presents an on - the - art methods in the composite of the learned features are selected using gini gain in algorithm design . _EOS_


In [16]:
prefix = 'to be or not to be' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

to be or not to be a useful tool to reduce the storage of all the three - dimensional data sets . in this work we propose a novel hierarchical model for image denoising by noise . we compare the performance of the world . in this paper , we propose a new method for learning a high level of the art in both cases , the proposed method is also an interactive visualization to promote the formation of the proposed method is able to achieve a better solution than the number of different sizes and rotation than other methods . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [17]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_tokens = 0
    corpora_log_perplexity = 0
    for line in lines:
        lexems = [UNK] + line.split() + [EOS]

        for i in range(1, len(lexems)):
            next_token = lexems[i]
            prefix = ' '.join(lexems[:i])
            rr = lm.get_next_token_prob(prefix, next_token)
            # if abs(rr) < 0.0000000001:
            #     print(rr, prefix, next_token)
            token_log_prob = np.log(rr)
            corpora_log_perplexity += max(token_log_prob, min_logprob)
            total_tokens += 1

    return np.exp( -1 / total_tokens * corpora_log_perplexity)

In [18]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/tmp/ipykernel_327866/3246578743.py:22: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(rr)


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [19]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/tmp/ipykernel_327866/3246578743.py:22: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(rr)


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259042902147072.00000


In [20]:
# whoops, it just blew up :)

In [21]:
import multiprocessing as mp

In [22]:
def helper(lm: NGramLanguageModel, lines, min_logprob=np.log(10 ** -50.)):
    total_tokens = 0
    corpora_log_perplexity = 0
    for line in lines:
        lexems = [UNK] + line.split() + [EOS]

        for i in range(1, len(lexems)):
            next_token = lexems[i]
            prefix = ' '.join(lexems[:i])
            token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
            corpora_log_perplexity += max(token_log_prob, min_logprob)
            total_tokens += 1

    return corpora_log_perplexity, total_tokens

def perplexity_parallel(lm: NGramLanguageModel, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """


    num_cores = mp.cpu_count()
    m = len(lines)
    partial_size = m // num_cores
    partials = []
    for i in range(num_cores):
        if i == num_cores - 1:
            partials.append((lm, lines[i*partial_size:]))
        else:
            partials.append((lm, lines[i*partial_size: (i+1)*partial_size]))
    with mp.Pool(num_cores) as pool:
        results = pool.starmap(helper, partials)

    total_tokens = 0
    corpora_log_perplexity = 0
    for log_p, toks in results:
        total_tokens += toks
        corpora_log_perplexity += log_p

    return np.exp( -1 / total_tokens * corpora_log_perplexity)


In [23]:
lm = NGramLanguageModel(n=1, lines=train_lines)

In [24]:
from timeit import default_timer
start = default_timer()
single_core_ppx = perplexity(lm, test_lines)
end = default_timer()
print('Single-Core PPX is:', single_core_ppx)
print('Took', end - start, 'seconds')

start = default_timer()
multi_core_ppx = perplexity_parallel(lm, test_lines)
end = default_timer()
print('Multi-Core PPX is:', multi_core_ppx)
print('Took', end - start, 'seconds')
assert np.allclose(single_core_ppx,multi_core_ppx), "Single-Core and Multi-Core PPX need to be equal"

/tmp/ipykernel_327866/3246578743.py:22: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(rr)


Single-Core PPX is: 1832.2313600472428
Took 9.08003034000285 seconds


/tmp/ipykernel_327866/3950676284.py:10: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
/tmp/ipykernel_327866/3950676284.py:10: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
/tmp/ipykernel_327866/3950676284.py:10: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
/tmp/ipykernel_327866/3950676284.py:10: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
/tmp/ipykernel_327866/3950676284.py:10: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
/tmp/ipykernel_327866/3950676284.py:10: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(lm.get_next_token_prob(prefix, next_token))
/tmp/ipykernel_327866/3950676284.py:10: 

Multi-Core PPX is: 1832.2313600158286
Took 1.4874122960027307 seconds


### Perplexity parallel is ~N times faster, when N = N_CPU
I have no idea why I did it, though, it was just interesting thing to try

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [25]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [26]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [27]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    start = default_timer()
    ppx = perplexity_parallel(lm, test_lines)
    end = default_timer()
    print("N = %i, Perplexity = %.5f, Took %i seconds" % (n, ppx, end - start))

N = 1, Perplexity = 977.67559, Took 2 seconds
N = 2, Perplexity = 470.48021, Took 3 seconds
N = 3, Perplexity = 3679.44765, Took 12 seconds


In [28]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [29]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=0.01):
        self.n = n
        counts_n = {}
        for i in range(1, n + 1):
            counts = count_ngrams(lines, i)
            counts_n[i] = counts

        self.vocab_n = {}
        for i in range(1, n + 1):
            vocab = set(token for token_counts in counts_n[i].values() for token in token_counts)
            self.vocab_n[i] = vocab

        self.probs_n = {}

        for i in range(1, n + 1):
            probs = defaultdict(Counter)
            for prefix in counts_n[i]:
                denom = sum(counts_n[i][prefix].values())
                # print(i, prefix, denom)
                if (i - 1) in self.probs_n:
                    for word, word_count in counts_n[i][prefix].items():
                        probs[prefix][word] = max(0, word_count - delta) / (denom)
                    # print('before norm', sum(probs[prefix].values()))
                    not_1 = sum(probs[prefix].values())
                    # print('Here:', i, prefix, denom, not_1)
                    diff = 0
                    for word, word_count in counts_n[i][prefix].items():
                        diff += self.probs_n[i-1][prefix[1:]][word]

                    inverse_lamb = 1 - not_1
                    lamb_prefix = inverse_lamb / diff

                    for word, word_count in counts_n[i][prefix].items():
                        probs[prefix][word] = max(0, word_count - delta) / (denom) + lamb_prefix * self.probs_n[i-1][prefix[1:]][word]
                        # print('after norm', sum(probs[prefix].values()))
                else:
                    for word, word_count in counts_n[i][prefix].items():
                        probs[prefix][word] = max(0, word_count - delta) / (denom)
                    not_1 = sum(probs[prefix].values())
                    # print(i, prefix, denom, not_1)
                    # print('Here:', i, prefix, denom, not_1)
                    for word, word_count in counts_n[i][prefix].items():
                        probs[prefix][word] /= not_1
                
            self.probs_n[i] = probs

        self.vocab = self.vocab_n[self.n]
        self.probs = self.probs_n[self.n]

    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

        
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))

In [30]:
dummy_lm = KneserNeyLanguageModel(dummy_lines, n=2)

In [32]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [33]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

/tmp/ipykernel_327866/3246578743.py:22: RuntimeWarning: divide by zero encountered in log
  token_log_prob = np.log(rr)


N = 1, Perplexity = 1832.19059
N = 2, Perplexity = 847691.39113
N = 3, Perplexity = 78463879218304.04688


#### OOOPS :)